<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import requests
import pandas as pd
import sqlite3
from google.colab import userdata

#Obtém as chaves de API do usuário.
def get_api_keys():
    return userdata.get('PUBLIC_KEY'), userdata.get('PRIVATE_KEY')

#Gera o hash MD5 necessário para autenticação na API / Timestamp fixo.
def generate_md5_hash(public_key, private_key):
    ts = "1"
    hash_input = f"{ts}{private_key}{public_key}"
    return hashlib.md5(hash_input.encode('utf-8')).hexdigest()

 #Faz uma requisição GET à API e retorna os dados em formato JSON.
def fetch_data_from_api(endpoint, public_key, hash_md5):
    base_url = "http://gateway.marvel.com"
    url = f"{base_url}/{endpoint}?ts=1&apikey={public_key}&hash={hash_md5}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

 #Cria as tabelas no banco de dados, caso não existam.
def create_tables(cursor):
    cursor.execute("""CREATE TABLE IF NOT EXISTS characters (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        description TEXT)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS comics (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        comic_title TEXT)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS events (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        description TEXT,
                        start DATE,
                        end DATE)""")

 #Insere os dados na tabela especificada do banco de dados.
def insert_data_into_db(cursor, data, table_name):
    for item in data:
        if table_name == 'characters':
            cursor.execute("""INSERT OR IGNORE INTO characters (id, name, description)
                              VALUES (?, ?, ?)""", (item["id"], item["name"], item["description"]))

        elif table_name == 'comics':
            cursor.execute("""INSERT INTO comics (comic_title)
                              VALUES (?)""", (item["title"],))

        elif table_name == 'events':
            cursor.execute("""INSERT OR IGNORE INTO events (title, description, start, end)
                              VALUES (?, ?, ?, ?)""", (item["title"], item["description"], item["start"], item["end"]))

#Função principal que executa todas as etapas do processo.
def main():
    try:
        public_key, private_key = get_api_keys()
        hash_md5 = generate_md5_hash(public_key, private_key)

        # Fetching data from Marvel API
        characters_data = fetch_data_from_api("v1/public/characters", public_key, hash_md5)["data"]["results"]
        comics_data = fetch_data_from_api("v1/public/comics", public_key, hash_md5)["data"]["results"]
        events_data = fetch_data_from_api("v1/public/events", public_key, hash_md5)["data"]["results"]

        # Conectando ao banco de dados SQLite
        with sqlite3.connect('marvel.db') as conn:
            cursor = conn.cursor()

            # Criando as tabelas no banco de dados
            create_tables(cursor)

            # Inserindo os dados nas tabelas
            insert_data_into_db(cursor, characters_data, 'characters')
            insert_data_into_db(cursor, comics_data, 'comics')
            insert_data_into_db(cursor, events_data, 'events')

        # Lendo as tabelas com pandas
        with sqlite3.connect('marvel.db') as conn:
            personagens_df = pd.read_sql_query("SELECT * FROM characters", conn)
            comics_df = pd.read_sql_query("SELECT * FROM comics", conn)
            events_df = pd.read_sql_query("SELECT * FROM events", conn)

        # Carrega os dados da tabela personagem como dataframe do Pandas
        df_Personagem = pd.DataFrame(personagens_df, columns = ['id',
                                                         'name',
                                                         'description'])

        # Carrega os dados da tabela quadrinhos como dataframe do Pandas
        df_Comics = pd.DataFrame(comics_df, columns = ['id',
                                                'comic_title'])

        # Carrega os dados da tabela eventos como dataframe do Pandas
        df_Events = pd.DataFrame(events_df, columns = ['title',
                                                'description',
                                                'start',
                                                'end'])

        print("Tabela de Personagem")
        display(df_Personagem)
        print("\n Tabela de Quadrinhos")
        display(df_Comics)
        print("\n Tabela de Eventos")
        display(df_Events)

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição à API: {e}")
    except sqlite3.DatabaseError as e:
        print(f"Erro no banco de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")


main()


Tabela de Personagem


,id,name,description
0,1009144,A.I.M.,AIM is a terrorist organization bent on destro...
1,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
2,1009148,Absorbing Man,
3,1009149,Abyss,
4,1009150,Agent Zero,
5,1010354,Adam Warlock,Adam Warlock is an artificially created human ...
6,1010699,Aaron Stack,
7,1010846,Aegis (Trey Rollins),
8,1010903,Abyss (Age of Apocalypse),
9,1011031,Agent X (Nijo),Originally a partner of the mind-altering assa...



 Tabela de Quadrinhos


,id,comic_title
0,1,Marvel Previews (2017)
1,2,Marvel Previews (2017)
2,3,Marvel Previews (2017)
3,4,Silver Surfer (1987)
4,5,Official Handbook of the Marvel Universe (2004...
...,...,...
235,236,Startling Stories: The Incorrigible Hulk (2004...
236,237,Official Handbook of the Marvel Universe (2004...
237,238,Gun Theory (2003) #4
238,239,Marvel Milestones (2005) #22



 Tabela de Eventos


,title,description,start,end
0,Acts of Vengeance!,Loki sets about convincing the super-villains ...,1989-12-10 00:00:00,2008-01-04 00:00:00
1,Age of Apocalypse,"In a twisted version of the world they knew, t...",1995-03-01 00:00:00,1996-06-01 00:00:00
2,Age of Ultron,In a 10-issue event written by Brian Michael B...,2013-03-06 00:00:00,2013-06-19 00:00:00
3,Age of X,In a strange reality where the X-Men never cam...,2011-01-26 00:00:00,2011-05-01 00:00:00
4,All-New All-Different Marvel,"This fall, prepare for an all-new, all-differe...",None,None
...,...,...,...,...
235,Civil War,After a horrific tragedy raises questions on w...,2006-07-01 00:00:00,2007-01-29 00:00:00
236,Civil War II,The drums of war beat for the Marvel Universe ...,None,None
237,Crossing,Iron Man betrays the Avengers to Kang at the c...,1995-09-10 00:00:00,1996-02-10 00:00:00
238,Dark Reign,Norman Osborn came out the hero of Secret Inva...,2008-12-01 00:00:00,2009-12-31 12:59:00
